# Bisección Global

In [ ]:
import os
import subprocess
import uproot
import numpy as np
import pandas as pd
from tqdm import tqdm

# Configuración de la simulación
sim_dir = r"C:\Users\conej\Documents\Universidad\Geant4\Projects\GIT-3\build\Release"
exec_file = "PCM.exe"
root_filename = "Sim0.root"
output_mac_path = os.path.join(sim_dir, "att.mac")
mac_template = """\
/run/initialize
/Pgun/Angle 0
/Pgun/Z -50 cm
/gun/energy {energy} keV
/det/t {thickness} nm
/run/reinitializeGeometry
/run/beamOn 10000
"""

# Función para crear y guardar el archivo .mac para la bisección
def create_mac_file(energy, thickness):
    mac_file_path = os.path.join(sim_dir, "bisection.mac")
    mac_content = mac_template.format(energy=energy, thickness=thickness)
    with open(mac_file_path, 'w') as f:
        f.write(mac_content)
    return mac_file_path

# Función para ejecutar la simulación
def run_simulation(mac_file_path):
    subprocess.run([os.path.join(sim_dir, exec_file), mac_file_path], cwd=sim_dir, check=True)

# Función para contar eventos en el archivo ROOT usando uproot y pandas
def count_events_from_root(root_filename):
    file_path = os.path.join(sim_dir, root_filename)
    if not os.path.isfile(file_path):
        return -1
    
    try:
        with uproot.open(file_path) as file:
            tree = file["G4_PCM"]
            df = tree.arrays(library="pd")
        
        if 'Energy' not in df.columns:
            return -1
        
        df_filtered = df[df['Energy'] > 0]
        hits = len(df_filtered)
        return hits

    except Exception as e:
        print(f"Error procesando {root_filename}: {e}")
        return -1

# Función para la optimización por bisección
def bisection_optimization(initial_energy, t_min, t_max, target_count=5000, tol=50, tol_min=1):
    while (t_max - t_min) > tol_min:
        t_mid = (t_min + t_max) / 2
        mac_file_path = create_mac_file(initial_energy, t_mid)
        run_simulation(mac_file_path)
        
        event_count = count_events_from_root(root_filename)
        if event_count == -1:
            break

        if event_count < (target_count - tol):
            t_max = t_mid
        elif event_count > (target_count + tol):
            t_min = t_mid
        else:
            return t_mid

    return (t_min + t_max) / 2

# Configuración del rango de energía y step size
energy_min = 20
energy_max = 100
step_size = 2

# Parámetros de bisección
t_min = 70
t_max = 90000000
target_count = 5000
tol = 50
tol_min = 1

# Generar valores de energía
energy_values = np.arange(energy_min, energy_max + step_size, step_size)

# DataFrame para almacenar los resultados
results = pd.DataFrame(columns=["Energy", "Optimal_t"])
previous_t = None

# Realizar optimización para cada valor de energía
for energy in tqdm(energy_values, desc="Optimizando", unit="energía"):
    if previous_t is not None:
        mac_file_path = create_mac_file(energy, previous_t)
        run_simulation(mac_file_path)
        
        event_count = count_events_from_root(root_filename)
        if event_count == -1:
            continue
        
        if abs(event_count - target_count) <= tol:
            results = pd.concat([results, pd.DataFrame({"Energy": [energy], "Optimal_t": [previous_t]})], ignore_index=True)
            continue
        else:
            optimal_t = bisection_optimization(energy, t_min, t_max, target_count, tol, tol_min)
            results = pd.concat([results, pd.DataFrame({"Energy": [energy], "Optimal_t": [optimal_t]})], ignore_index=True)
            previous_t = optimal_t
    else:
        optimal_t = bisection_optimization(energy, t_min, t_max, target_count, tol, tol_min)
        results = pd.concat([results, pd.DataFrame({"Energy": [energy], "Optimal_t": [optimal_t]})], ignore_index=True)
        previous_t = optimal_t

# Crear el archivo att.mac para la ejecución final
output_mac_path = os.path.join(sim_dir, "att.mac")
with open(output_mac_path, "w") as mac_file:
    mac_file.write("/run/initialize\n")

    for _, row in results.iterrows():
        energy = row["Energy"]
        thickness = row["Optimal_t"]
        
        mac_file.write(f"/gun/energy {energy} keV\n")
        mac_file.write("/Pgun/Angle 0\n")
        mac_file.write("/Pgun/Z -50 cm\n")
        mac_file.write(f"/det/t {thickness} nm\n")
        mac_file.write("/run/reinitializeGeometry\n")
        mac_file.write("/run/beamOn 1000000\n\n")

print(f"Archivo macro generado en: {output_mac_path}")

# Ejecutar Geant4 con el archivo att.mac
subprocess.run([os.path.join(sim_dir, exec_file), output_mac_path], cwd=sim_dir, check=True)

# Calcular coeficiente de atenuación
# Parámetro de densidad (variable)
density_value = 8.94  # Cambia este valor según sea necesario

# Función para calcular el coeficiente de atenuación usando valores de grosor específicos
def calculate_coeff(thickness, hits):
    coeff = (-1 / thickness) * np.log(hits / 1000000) / density_value
    return coeff

# Procesar todos los archivos ROOT y calcular coeficientes
data = []
for energy in energy_values:
    root_filename = f"Sim{int((energy - energy_min) / step_size)}.root"  # Asumir que los nombres de archivos siguen un patrón
    thickness = results.loc[results["Energy"] == energy, "Optimal_t"].values[0] / 1e7  # Convertir a cm
    hits = count_events_from_root(root_filename)
    
    if hits > 0:
        coeff = calculate_coeff(thickness, hits)
        data.append([energy, coeff])
    else:
        print(f"Archivo {root_filename} no pudo ser procesado correctamente.")

# Convertir los resultados a un DataFrame
df_results = pd.DataFrame(data, columns=["Energy (keV)", "MAC"])

# Guardar los resultados en un archivo CSV
output_csv = os.path.join(sim_dir, "attenuation_mac.csv")
df_results.to_csv(output_csv, index=False)

print(f"Archivo CSV guardado en: {output_csv}")


Optimizando:   0%|          | 0/41 [00:00<?, ?energía/s]C:\Users\conej\AppData\Local\Temp\ipykernel_25088\591246493.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame({"Energy": [energy], "Optimal_t": [optimal_t]})], ignore_index=True)
Optimizando: 100%|██████████| 41/41 [03:38<00:00,  5.33s/energía]


Archivo macro generado en: C:\Users\conej\Documents\Universidad\Geant4\Projects\GIT-3\build\Release\att.mac
Archivo CSV guardado en: C:\Users\conej\Documents\Universidad\Geant4\Projects\GIT-3\build\Release\nt1mac.csv


In [ ]:
import os
import subprocess
import uproot
import numpy as np
import pandas as pd
from tqdm import tqdm
import logging

# Configuración del logger
logging.basicConfig(filename='simulation.log', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Configuración de la simulación
sim_dir = r"C:\Users\conej\Documents\Universidad\Geant4\Projects\GIT-3\build\Release"
exec_file = "PCM.exe"
root_filename = "Sim0.root"
output_mac_path = os.path.join(sim_dir, "att.mac")
mac_template = """\
/run/initialize
/Pgun/Angle 0
/Pgun/Z -50 cm
/gun/energy {energy} keV
/det/t {thickness} nm
/run/reinitializeGeometry
/run/beamOn 10000
"""

# Función para crear y guardar el archivo .mac para la bisección
def create_mac_file(energy, thickness):
    mac_file_path = os.path.join(sim_dir, "bisection.mac")
    mac_content = mac_template.format(energy=energy, thickness=thickness)
    with open(mac_file_path, 'w') as f:
        f.write(mac_content)
    return mac_file_path

# Función para ejecutar la simulación
def run_simulation(mac_file_path):
    try:
        subprocess.run([os.path.join(sim_dir, exec_file), mac_file_path], cwd=sim_dir, check=True)
        logging.info(f"Simulación ejecutada con el archivo: {mac_file_path}")
    except subprocess.CalledProcessError as e:
        logging.error(f"Error en la simulación: {e}")
        print(f"Error en la simulación: {e}")

# Función para contar eventos en el archivo ROOT
def count_events_from_root(root_filename):
    file_path = os.path.join(sim_dir, root_filename)
    if not os.path.isfile(file_path):
        logging.warning(f"Archivo no encontrado: {file_path}")
        return -1
    
    try:
        with uproot.open(file_path) as file:
            tree = file["G4_PCM"]
            df = tree.arrays(library="pd")
        
        if 'Energy' not in df.columns:
            logging.warning("Columna 'Energy' no encontrada en el DataFrame.")
            return -1
        
        df_filtered = df[df['Energy'] > 0]
        hits = len(df_filtered)
        logging.info(f"Conteo de eventos: {hits} en {root_filename}")
        return hits

    except Exception as e:
        logging.error(f"Error procesando {root_filename}: {e}")
        return -1

# Función para la optimización por bisección
def bisection_optimization(initial_energy, t_min, t_max, target_count=5000, tol=50, tol_min=1):
    while (t_max - t_min) > tol_min:
        t_mid = (t_min + t_max) / 2
        mac_file_path = create_mac_file(initial_energy, t_mid)
        run_simulation(mac_file_path)
        
        event_count = count_events_from_root(root_filename)
        if event_count == -1:
            break

        if event_count < (target_count - tol):
            t_max = t_mid
        elif event_count > (target_count + tol):
            t_min = t_mid
        else:
            return t_mid

    return (t_min + t_max) / 2

# Generar valores de energía y DataFrame para almacenar resultados
energy_values = np.arange(energy_min, energy_max + step_size, step_size)
results = pd.DataFrame(columns=["Energy", "Optimal_t"])
previous_t = None

# Realizar optimización para cada valor de energía
for energy in tqdm(energy_values, desc="Optimizando", unit="energía"):
    if previous_t is not None:
        mac_file_path = create_mac_file(energy, previous_t)
        run_simulation(mac_file_path)
        
        event_count = count_events_from_root(root_filename)
        if event_count == -1:
            continue
        
        if abs(event_count - target_count) <= tol:
            results = pd.concat([results, pd.DataFrame({"Energy": [energy], "Optimal_t": [previous_t]})], ignore_index=True)
            continue
        else:
            optimal_t = bisection_optimization(energy, t_min, t_max, target_count, tol, tol_min)
            results = pd.concat([results, pd.DataFrame({"Energy": [energy], "Optimal_t": [optimal_t]})], ignore_index=True)
            previous_t = optimal_t
    else:
        optimal_t = bisection_optimization(energy, t_min, t_max, target_count, tol, tol_min)
        results = pd.concat([results, pd.DataFrame({"Energy": [energy], "Optimal_t": [optimal_t]})], ignore_index=True)
        previous_t = optimal_t

# Crear el archivo att.mac para la ejecución final
output_mac_path = os.path.join(sim_dir, "att.mac")
with open(output_mac_path, "w") as mac_file:
    mac_file.write("/run/initialize\n")

    for _, row in results.iterrows():
        energy = row["Energy"]
        thickness = row["Optimal_t"]
        
        mac_file.write(f"/gun/energy {energy} keV\n")
        mac_file.write("/Pgun/Angle 0\n")
        mac_file.write("/Pgun/Z -50 cm\n")
        mac_file.write(f"/det/t {thickness} nm\n")
        mac_file.write("/run/reinitializeGeometry\n")
        mac_file.write("/run/beamOn 1000000\n\n")

logging.info(f"Archivo macro generado en: {output_mac_path}")

# Ejecutar Geant4 con el archivo att.mac
try:
    subprocess.run([os.path.join(sim_dir, exec_file), output_mac_path], cwd=sim_dir, check=True)
    logging.info(f"Ejecución final de Geant4 completada con {output_mac_path}")
except subprocess.CalledProcessError as e:
    logging.error(f"Error en la ejecución final: {e}")

# Calcular coeficiente de atenuación
density_value = 8.96  # Cambia este valor según sea necesario

def calculate_coeff(thickness, hits):
    coeff = (-1 / thickness) * np.log(hits / 1000000) / density_value
    return coeff

# Procesar todos los archivos ROOT y calcular coeficientes
data = []
for energy in energy_values:
    root_filename = f"Sim{int((energy - energy_min) / step_size)}.root"
    thickness = results.loc[results["Energy"] == energy, "Optimal_t"].values[0] / 1e7  # Convertir a cm
    hits = count_events_from_root(root_filename)
    
    if hits > 0:
        coeff = calculate_coeff(thickness, hits)
        data.append([energy, coeff])
    else:
        logging.warning(f"Archivo {root_filename} no pudo ser procesado correctamente.")

# Convertir los resultados a un DataFrame
df_results = pd.DataFrame(data, columns=["Energy (keV)", "MAC"])

# Guardar los resultados en un archivo CSV
output_csv = os.path.join(sim_dir, "attenuation_mac.csv")
df_results.to_csv(output_csv, index=False)

logging.info(f"Archivo CSV guardado en: {output_csv}")


Optimizando:   0%|          | 0/41 [00:00<?, ?energía/s]C:\Users\conej\AppData\Local\Temp\ipykernel_25088\31491742.py:113: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame({"Energy": [energy], "Optimal_t": [optimal_t]})], ignore_index=True)
Optimizando: 100%|██████████| 41/41 [10:31<00:00, 15.41s/energía]
